<a href="https://colab.research.google.com/github/massenergize/rad/blob/hasha_refactor/merge-data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Understand the python environment

The google colab environment has a bunch of packages installed.  It's not using conda environments.

In [1]:
! conda list

/bin/bash: conda: command not found


In [2]:
! pip list

Package                       Version        
----------------------------- ---------------
absl-py                       0.10.0         
alabaster                     0.7.12         
albumentations                0.1.12         
altair                        4.1.0          
argon2-cffi                   20.1.0         
asgiref                       3.3.1          
astor                         0.8.1          
astropy                       4.1            
astunparse                    1.6.3          
async-generator               1.10           
atari-py                      0.2.6          
atomicwrites                  1.4.0          
attrs                         20.3.0         
audioread                     2.1.9          
autograd                      1.3            
Babel                         2.9.0          
backcall                      0.2.0          
beautifulsoup4                4.6.3          
bleach                        3.2.1          
blis                          0.4.

# RENEWABLES ACTION DATASET ANNOTATIONS 


## Electric Vehicles
* Source: Center for Sustainable Energy (2020). Massachusetts Department of  Energy Resources Massachusetts Offers Rebates for Electric Vehicles, Rebate Statistics.  
* Retrieved 09/08/2020 from: https://mor-ev.org/program-statistics
* Data last updated 08/21/2020. Data date range includes 06/19/2014 - 08/15/2020.
* Sectors: Residential.


## Res ASHP

* Source: Massachusetts Clean Energy Center (2020). Air Source Heat Pump Program - Residential Projects.
* Retrieved 09/08/2020 from: http://files-cdn.masscec.com/ResidentialASHPProjectDatabase%2011.4.2019.xlsx
* Data last updated 11/04/2019. Data date range includes 12/26/2014 - 10/23/2019.
* Sectors: Residential.


## GSHP

* Source: Massachusetts Clean Energy Center (2020). Ground Source Heat Pump Program - Residential & Small-Scale Projects Database.
* Retrieved 09/08/2020 from: http://files-cdn.masscec.com/get-clean-energy/govt-np/clean-heating-cooling/ResidentialandSmallScaleGSHPProjectDatabase.xlsx 
* Data last updated June 2020. Data date range includes 01/02/2015 - 06/09/2020.
* Sectors: Residential, Small Commercial.


# Mount input data files from Google Drive

Instructions from [this tutorial](https://colab.research.google.com/notebooks/io.ipynb)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!ls /content/drive

MyDrive  Shareddrives


In [8]:
!ls /content/drive/Shareddrives/MEinternal-DataWG/RAD\ Renewable\ Actions\ Data

'05-04-2020 Data Group Agenda.gdoc'
 Analyses
 Code
'Data Downloads'
'Data Fields Overview.xlsx'
'Data Not Yet Included'
 MassData
 merge-data-AH.ipynb
 merge-data-v3.ipynb
'[old] merge-data.ipynb'
'Renewable Actions Dataset Instructions.gdoc'
'Renewables Action Dataset Annotations.gdoc'
 SAMPLE-all-actions-data.xlsx
'Untitled document.gdoc'
'Zip Code Community.xlsx'
'Zip Code Muni Name.drawio'


# Work with the data

In [57]:
import pandas as pd
import os
import re

In [18]:
data_dir = "/content/drive/Shareddrives/MEinternal-DataWG/RAD Renewable Actions Data"
data_files = {
    "zip_code_community": os.path.join(data_dir, "Zip Code Community.xlsx"),
    "residental_ashp_project_database": os.path.join(data_dir, "ResidentialASHPProjectDatabase 11.4.2019.xlsx"),
    "pv_in_pts_website": os.path.join(data_dir, "PVinPTSwebsite.xlsx")
}
data_files

{'pv_in_pts_website': '/content/drive/Shareddrives/MEinternal-DataWG/RAD Renewable Actions Data/PVinPTSwebsite.xlsx',
 'residental_ashp_project_database': '/content/drive/Shareddrives/MEinternal-DataWG/RAD Renewable Actions Data/ResidentialASHPProjectDatabase 11.4.2019.xlsx',
 'zip_code_community': '/content/drive/Shareddrives/MEinternal-DataWG/RAD Renewable Actions Data/Zip Code Community.xlsx'}

Standardize and validate zipcodes

In [37]:
#PTS solar data next is the important one, tracking data

# Load Zip Code Municipality -- 
# @params: file_name, column_name
zipcodes = pd.read_excel(data_files["zip_code_community"], 'Villages to Muni with Zip')
municipalities = pd.read_excel(data_files["zip_code_community"], '351 Mass Munis')

# One thing for these datasets is that it's possible to read all the ZipCode fields as strings, and in that case it would not get rid of the '0' prefix in front of all Zip Codes
f_ashp = pd.read_excel(data_files["residental_ashp_project_database"], 'Sheet1', skiprows=3)
df_ashp = f_ashp.drop([0]) #remove first null row for formatting purposes

#Read PVinPTS Data
df_pv = pd.read_excel(data_files["pv_in_pts_website"])

#name_of_dataframe = pd.read_excel('nameoffile.xlsx')

In [20]:
df_ashp.head()

,Date Rebate Payment Approved by MassCEC,Site City/Town,Site Zip Code,Installer Company Name,Heating Fuel Being Replaced,Cooling Type Being Replaced,# of Outdoor Units,# of Indoor Units,Capacity of Heat Pumps at 5°F,Single- Head Heat Pump #1,Single- Head Heat Pump #2,Single- Head Heat Pump #3,Multi-Head Heat Pump #1,Multi-Head Heat Pump #2,Multi-Head Heat Pump #3,Total System Costs,Receiving an Income-Based Adder?,Rebate Amount
1,2019-10-23,CENTERVILLE,2632,"Seaside Gas Service, Inc",Natural Gas,None,1,3,25000.761761,NaN,NaN,NaN,NaN,NaN,NaN,11575,0,1302.08
2,2019-10-23,NORTHFIELD,1360,Arctic Refrigeration,Pellet Stove,2 window units,1,1,20300.000000,NaN,NaN,NaN,NaN,NaN,NaN,4050,0,625
3,2019-10-16,West Tisbury,2575,Nelson Mechanical Design Inc,Propane,Window Unit(s),2,-,57200.400000,,,,Mitsubishi MXZ-3C30NAHZ2,Mitsubishi MXZ-3C30NAHZ2,NaN,25560,No - Not Applicable,2000
4,2019-10-16,FITCHBURG,1420,Royal Steam Heater Co.,Natural Gas,Window fan,4,15,92997.920266,NaN,NaN,NaN,NaN,NaN,NaN,44352,0,6200
5,2019-10-09,Haverhill,01835,Climate Zone,Oil,Window Unit(s),1,NaN,28600.000000,0,0,0,Mitsubishi-MXZ-3C30NAHZ2,0,0,13700,No - Not Applicable,1191.67


Looks like data describing individual installation projects. 

# Cleaning up zipcodes

Clean zipcodes by

1. cast each entry as a string
2. stripping whitespace from left and right
3. leftpad with zeros to a minimum width of five

In [65]:
df_ashp['zip code'] = df_ashp['Site Zip Code'].astype(str).str.strip().str.zfill(5)

In [70]:
# Valid zipcodes are 5 digits 0-9, followed by an optional dash and 4 more digits.
valid_zipcode_regex = r"^[0-9]{5}(?:-[0-9]{4})?$"

In [69]:
# Check records that don't match the zipcode pattern
df_ashp[~df_ashp['zip code'].str.match(valid_zipcode_regex)]['zip code']

1122       01545-
1590    019081047
1606        0212y
Name: zip code, dtype: object

The last record looks like a typo that should be dropped.  The first two look like the correct zipcode can be manually guessed.

In [25]:
df_ashp["Site Zip Code"].head()

1     2632
2     1360
3     2575
4     1420
5    01835
Name: Site Zip Code, dtype: object

In [ ]:
#Not correct, weird issues with indexes and assigning appropriate values
def formatZipCode(df, columnName):
    for index, row in df.iterrows():
        if not isinstance(index, int):
            continue
        if index >= len(df):
            print(index)
            continue
        x = df[columnName].iloc[index]
        print(x)

        if         print(x)
            df.at[index, columnName] = x[:5]
        elif         print(x)
            if len(str(x)) == 4:
                df.at[index, columnName] = str('0' + str(x))
                # df.at[index+1, columnName] = 88888
            else: 
                print("Odd formatting with ", x)

formatZipCode(df_ashp, 'Site Zip Code')

print(df_ashp['Site Zip Code'])

07
01879
01464
02559
02667
02019
02360
01301
02540
02169
01970
01532
01030
01095
01772
02119
01772
02038
02038
02132
01752
02364
01104
02338
02539
02675
01803
01876
02664
01073
01913
01760
02136
02129
01844
02631
01062
01904
02563
02558
02563
02169
01027
01373
02631
02667
02663
02633
02038
01810
01702
02081
01701
02171
01002
02664
02030
01516
01364
02360
02136
01826
01569
01604
01604
02375
02130
02633
02648
02130
01721
01462
02050
02191
02332
01569
02035
01128
02021
01778
02639
02657
02739
01360
01754
01906
02081
01331
02635
01331
01824
01027
02648
01760
02301
02666
01588
01752
02645
02129
01376
02421
02494
02052
02639
02143
01519
01862
02481
02563
02081
02360
02364
02155
01151
02144
02644
02563
02664
01344
02633
02359
02631
01810
01756
01095
01938
02459
01810
01930
02081
01301
01702
02461
02061
02536
02631
01507
01095
01082
01001
02642
02460
02067
02129
01702
02641
02453
02072
02136
01701
02136
02359
01760
01950
02653
02066
02631
01543
01373
01070
02379
02575
02332
02638
02476
02420
0

In [ ]:
#Checks that the data's town name is present in the 351 mass munis list or villages to muni with zip list. 
def matchMuniName(siteCityName, siteZipCode):
    x = municipalities.loc[municipalities['Municipality'] == siteCityName]
    if not x.empty:
        if x.size > 1:
            print("There are multiple municipalities for,", siteCityName)
        return x.iloc[0]['City']
    else:
        y = zipcodes.loc[zipcodes['City'] == siteCityName]
        if y.size > 1: 
            print("There are multiple municipalities for", siteCityName)
        return y.iloc[0]['City']
        #also use zip codes
    return "NF"

matchMuniName("Florence", '02138')

There are multiple municipalities for Florence


'Florence'

In [ ]:
#Checks that the data's town name is present in the 351 mass munis list or villages to muni with zip list. 
def matchMuniName(siteCityName, siteZipCode):
    x = municipalities.loc[municipalities['Municipality'] == siteCityName]
    if not x.empty:
        # print(x.values)
        return x.values.astype('str')
    else:
        y = zipcodes.loc[zipcodes['City'] == siteCityName]
        print(y)
        return y
        #also use zip codes
    return "NF"

matchMuniName("Boston", '02138')

array([['Boston']], dtype='<U6')

In [ ]:
#Implement and use the predefined matchMuniName function. 
def processZipCode(df, zipCodeColumnName, townColumnName, newMuniName):
    counter = 0
    for index,row in df.iterrows():
        if row[townColumnName]:
            if not df.loc[df[townColumnName].str.lower() == row['Site City/Town'].lower()].empty:
                print(row['Site City/Town'])
        #first use the 351 Mass Munis (the municipalities dataframe)
        #then use the Villages to Munu with Zip (the zipcodes data frame) to match if value is not present in mass munis. 
            
            #else: if the Site City/Town is not in the municipality list, then choose a different 

        #if there is a zip code 
            #if not a zip code 
                #check multi zip code
                    #find address
                #use matchMuniName()

processZipCode(df_ashp, "", "", "")

In [ ]:
#Format Technology

df_ashp = df_ashp.assign(Technology = 'Air Source Heat Pumps')
#Do the Same for other data sets

print(df_ashp)

Date Rebate Payment Approved by MassCEC  Site City/Town Site Zip Code  \
1                                  2019-10-23     CENTERVILLE          2632   
2                                  2019-10-23      NORTHFIELD          1360   
3                                  2019-10-16    West Tisbury          2575   
4                                  2019-10-16       FITCHBURG          1420   
5                                  2019-10-09       Haverhill         01835   
...                                       ...             ...           ...   
19960                              2015-01-14         Bedford          1730   
19961                              2015-01-08     Westborough          1581   
19962                              2015-01-08  Yarmouth Port           2675   
19963                              2015-01-06     LEOMINSTER           1453   
19964                              2014-12-26         Amherst          1002   

                 Installer Company Name Heating Fuel Bein

In [ ]:
#Date and Time Formatting

def processDates(df, dateColumn, yearName, monthName):
    for index, row in df.iterrows():
        date = row[dateColumn]
        row[yearName] = date.year
        row[monthName] = date.month
    #select the year, month from these columns

processDates(df_ashp,'Date Rebate Payment Approved by MassCEC', 'Year', 'Month')

In [ ]:
#Quantity: Use Pandas df['Application Number'].unique() to find all unique values

print(df_ashp['Site City/Town'].unique())

['CENTERVILLE' 'NORTHFIELD' 'West Tisbury' ... 'Shelburne Center'
 'Erving ' 'N Carver ']


In [ ]:
#Rebates

#Assign rebates directy from the reabtes in the ASHP data -- read all as integer or dollar values

In [ ]:
#"Snap to Grid"
# Match Column names and copy over data


In [ ]:
#Data Fields for Final Dataset 
dataFields = ['Municipality', 'Zip Code', 'Tech', 'Year', 'Month', 'Quantity', 'Average Cost',
    'Total Cost', 'Total Rebates', 'Average Rebate', 'Count Income-Eligible']

df_final = pd.DataFrame(columns=dataFields)

print(df_final)

#to_csv